In [5]:
import pandas as pd
from datetime import datetime, timedelta, date
import io

montana_df = pd.read_csv("data/states/Montana/montana.csv", error_bad_lines=True)
montana_df = montana_df[(montana_df['county'] > 0) & (montana_df['county'] < 112)]
county_pop_df = pd.read_csv("data/states/county_pop.csv", encoding = "utf-8",error_bad_lines=True)
county_pop_df = county_pop_df[county_pop_df['STNAME'] == 'Montana']
#print(montana_df)
#print(county_pop_df)

montana_constant_df = montana_df.copy()
montana_constant_df['state_FIPS'] = 30
montana_constant_df['population_adjustment'] = 'Constant'
montana_constant_df['pop'] = 0
montana_constant_df["total_deaths_per100k"] = 0
montana_constant_df["excess_deaths_per100k"] = 0.0
montana_constant_df["age_adjusted_deaths_lt_65_per100k"] = 0
montana_constant_df["age_adjusted_deaths_gte_65_per100k"] = 0
montana_constant_df["excess_age_adjusted_deaths_lt_65_per100k"] = 0.0
montana_constant_df["excess_age_adjusted_deaths_gte_65_per100k"] = 0.0
county_2017_pop= dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
pop_100k = 100000.0
for index, row in montana_constant_df.iterrows():
    county = row['county']
    montana_constant_df.at[index,'pop'] = county_2017_pop[county]
    montana_constant_df.at[index, 'total_deaths_per100k'] = (row["total_deaths"] *pop_100k )/(county_2017_pop[county]* 1.0)
    montana_constant_df.at[index, "age_adjusted_deaths_lt_65_per100k"] = (row["less_than_65_deaths"] * pop_100k)/(county_2017_pop[county]* 1.0)
    montana_constant_df.at[index, "age_adjusted_deaths_gte_65_per100k"] = (row["greater_than_65_deaths"] * pop_100k)/(county_2017_pop[county]* 1.0)

avg_deaths_df = montana_constant_df[montana_constant_df['year'] < 2020].groupby(['county', 'week_num'])['total_deaths_per100k','age_adjusted_deaths_lt_65_per100k','age_adjusted_deaths_gte_65_per100k'].mean().reset_index()

for index, row in montana_constant_df.iterrows():
    year = row['year']
    county = row['county']
    week = row['week_num']
    if year == 2020:
        montana_constant_df.at[index, "excess_deaths_per100k"] = row['total_deaths_per100k'] - (avg_deaths_df[(avg_deaths_df['county'] == county) & (avg_deaths_df['week_num'] == week)]['total_deaths_per100k'])       
        montana_constant_df.at[index, "excess_age_adjusted_deaths_lt_65_per100k"] = row['age_adjusted_deaths_lt_65_per100k'] - (avg_deaths_df[(avg_deaths_df['county'] == county) & (avg_deaths_df['week_num'] == week)]['age_adjusted_deaths_lt_65_per100k'])       
        montana_constant_df.at[index, "excess_age_adjusted_deaths_gte_65_per100k"] = row['age_adjusted_deaths_gte_65_per100k'] - (avg_deaths_df[(avg_deaths_df['county'] == county) & (avg_deaths_df['week_num'] == week)]['age_adjusted_deaths_gte_65_per100k'])       
        


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [6]:
county_year_dict = dict()
county_year_dict[2014] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2014))
county_year_dict[2015] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2015))
county_year_dict[2016] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2016))
county_year_dict[2017] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
county_year_dict[2018] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2018))
county_year_dict[2019] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2019))


montana_lagged_df = montana_df.copy()
montana_lagged_df['state_FIPS'] = 30
montana_lagged_df['population_adjustment'] = 'Lagged'
montana_lagged_df['pop'] = 0
montana_lagged_df["total_deaths_per100k"] = 0
montana_lagged_df["excess_deaths_per100k"] = 0.0
montana_lagged_df["age_adjusted_deaths_lt_65_per100k"] = 0
montana_lagged_df["age_adjusted_deaths_gte_65_per100k"] = 0
montana_lagged_df["excess_age_adjusted_deaths_lt_65_per100k"] = 0.0
montana_lagged_df["excess_age_adjusted_deaths_gte_65_per100k"] = 0.0
pop_100k = 100000.0


for index, row in montana_lagged_df.iterrows():
    year = row['year'] - 1  
    county = row['county']
    county_pop_dict = county_year_dict[year]
    montana_lagged_df.at[index,'pop'] = county_pop_dict[county]
    montana_lagged_df.at[index, 'total_deaths_per100k'] = (row["total_deaths"] *pop_100k )/(county_pop_dict[county]* 1.0)
    montana_lagged_df.at[index, "age_adjusted_deaths_lt_65_per100k"] = (row["less_than_65_deaths"] * pop_100k)/(county_pop_dict[county]* 1.0)
    montana_lagged_df.at[index, "age_adjusted_deaths_gte_65_per100k"] = (row["greater_than_65_deaths"] * pop_100k)/(county_pop_dict[county]* 1.0)

avg_lagged_deaths_df = montana_lagged_df[montana_lagged_df['year'] < 2020].groupby(['county', 'week_num'])['total_deaths_per100k','age_adjusted_deaths_lt_65_per100k','age_adjusted_deaths_gte_65_per100k'].mean().reset_index()

for index, row in montana_lagged_df.iterrows():
    year = row['year']
    county = row['county']
    week = row['week_num']
    if year == 2020:
        montana_lagged_df.at[index, "excess_deaths_per100k"] = row['total_deaths_per100k'] - (avg_lagged_deaths_df[(avg_lagged_deaths_df['county'] == county) & (avg_lagged_deaths_df['week_num'] == week)]['total_deaths_per100k'])       
        montana_lagged_df.at[index, "excess_age_adjusted_deaths_lt_65_per100k"] = row['age_adjusted_deaths_lt_65_per100k'] - (avg_lagged_deaths_df[(avg_lagged_deaths_df['county'] == county) & (avg_lagged_deaths_df['week_num'] == week)]['age_adjusted_deaths_lt_65_per100k'])       
        montana_lagged_df.at[index, "excess_age_adjusted_deaths_gte_65_per100k"] = row['age_adjusted_deaths_gte_65_per100k'] - (avg_lagged_deaths_df[(avg_lagged_deaths_df['county'] == county) & (avg_lagged_deaths_df['week_num'] == week)]['age_adjusted_deaths_gte_65_per100k'])       

montana_merged_df = pd.concat([montana_constant_df, montana_lagged_df])
montana_merged_df = montana_merged_df.sort_values(['county', 'week_num', 'population_adjustment','year'])
#montana_panel_df = pd.DataFrame(columns=['State_FIPS','County_FIPS', 'Year','Week', 'Population_Adjustment','Deaths_Per100k',
#                                'Excess_Deaths_Per100k' 'Age_Adjusted_Deaths_LT_65_Per100k','Age_Adjusted_Deaths_GTE_65_Per100k', 'POP'])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [7]:
montan_panel_df = pd.DataFrame(columns=['State_FIPS','County_FIPS','Year', 'Week','Population_Adjustment', 
                                        'Deaths_Per100k', 'Excess_Deaths_Per100k','Age_Adjusted_Deaths_LT_65_Per100k',
                                        'Age_Adjusted_Deaths_GTE_65_Per100k','Excess_Age_Adjusted_Deaths_LT_65_Per100k','Excess_Age_Adjusted_Deaths_GTE_65_Per100k'])        
montan_panel_df['State_FIPS']  = montana_merged_df['state_FIPS']
montan_panel_df['County_FIPS'] = montana_merged_df['county']
montan_panel_df['Year']  = montana_merged_df['year']
montan_panel_df['Week'] = montana_merged_df['week_num']
montan_panel_df['Population_Adjustment'] = montana_merged_df['population_adjustment']
montan_panel_df['Deaths_Per100k'] = montana_merged_df['total_deaths_per100k']
montan_panel_df['Excess_Deaths_Per100k'] = round(montana_merged_df['excess_deaths_per100k'],4)
montan_panel_df['Age_Adjusted_Deaths_LT_65_Per100k'] = montana_merged_df['age_adjusted_deaths_lt_65_per100k']
montan_panel_df['Age_Adjusted_Deaths_GTE_65_Per100k'] = montana_merged_df['age_adjusted_deaths_gte_65_per100k']
montan_panel_df['Excess_Age_Adjusted_Deaths_LT_65_Per100k'] = round(montana_merged_df['excess_age_adjusted_deaths_lt_65_per100k'],4)
montan_panel_df['Excess_Age_Adjusted_Deaths_GTE_65_Per100k'] = round(montana_merged_df['excess_age_adjusted_deaths_gte_65_per100k'],4)

In [8]:
montan_panel_df.to_csv('montan_panel.csv',index=False)